## Soulbound

[Official rules.](https://cubicle7games.com/blog/soulbound-how-tests-work)

[StackExchange question.](https://math.stackexchange.com/questions/4403443/probability-of-rolling-n-dice-that-are-each-are-greater-than-or-equal-to-x-w)

[Reddit question.](https://www.reddit.com/r/RPGdesign/comments/zkduz7/understanding_the_probability_of_rolling_a_number/)

* Roll a pool of d6.
* Each die >= the DN is a success.
* Focus points can be used to increase the value of a die by 1 each.

Which dice should be upgraded? Since each focus point can be distributed independently,
you can't go wrong by upgrading the dice that are the closest to the threshold first.
Therefore, we can consider the dice in descending order and upgrade dice greedily.

In [1]:
%pip install icepool

from icepool import d6, MultisetEvaluator, Order

class SoulboundEvaluator(MultisetEvaluator):
    def __init__(self, dn, focus):
        self._dn = dn
        self._focus = focus
    
    def next_state(self, state, outcome, count):
        if state is None:
            success, focus = 0, self._focus
        else:
            success, focus = state
        
        if outcome >= self._dn:
            success += count
        else:
            # Upgrade as many dice as possible to successes.
            focus_per_die = self._dn - outcome
            focus_dice = min(focus // focus_per_die, count)
            success += focus_dice
            focus -= focus_per_die * focus_dice
        return success, focus
    
    def final_outcome(self, final_state):
        # Return just the number of successes.
        success, focus = final_state
        return success
        
    def order(self, *_):
        # See outcomes in descending order.
        return Order.Descending

result = SoulboundEvaluator(dn=5, focus=3).evaluate(d6.pool(10))
print(result)
print('Chance of at least 4 successes:')
print(result >= 4)

Die with denominator 60466176

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |        1 |   0.000002% |
|       1 |    64188 |   0.106155% |
|       2 |   926855 |   1.532849% |
|       3 |  4573072 |   7.563025% |
|       4 | 11264720 |  18.629787% |
|       5 | 16259004 |  26.889420% |
|       6 | 14777568 |  24.439396% |
|       7 |  8650560 |  14.306445% |
|       8 |  3197184 |   5.287558% |
|       9 |   686720 |   1.135709% |
|      10 |    66304 |   0.109655% |


Chance of at least 4 successes:
Die with denominator 60466176

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| False   |  5564116 |   9.202031% |
| True    | 54902060 |  90.797969% |


